In [1]:
from glob import glob
from os import path
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pandas as pd

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_output import run_solver_output

The solver is a single Python script, contained in `solver/solver.py`.
It takes the following parameters:
* `-m` to specify the model name (`base`, `fixed`, `partflex`, or `flex`).
* `-i` to specify the location of the instance file.
* `-c` to specify the outsourcing cost multiplier (parameter OC in the paper).
* `-r` to specify the regional bound multiplier (parameter RM in the paper).
* `-g` to specify the global bound multiplier (parameter GM in the paper).
* `-u` to specify the maximum number of shift start times for the `partflex` model (parameter $\mu$ in the paper).
* `-o` to specify the location of the JSON solution file produced by the solver.

In [3]:

file = 'berlin_db=0.50_dt=atend.json'

instance_ = f"../instances/{file}"

run_solver_output(model='flex', instance=instance_, outsourcing_cost_multiplier=1.2, regional_multiplier=.75, global_multiplier=.6)


in instance
in solver
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14684 rows, 22728 columns and 29986 nonzeros
Model fingerprint: 0x3f258453
Variable types: 14160 continuous, 8568 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 550.8400081
Found heuristic solution: objective 550.8399961
Presolve removed 13812 rows and 14966 columns
Presolve time: 0.03s
Presolved: 872 rows, 7762 columns, 16019 nonzeros
Found heuristic solution: objective 550.8400000
Variable types: 673 continuous, 7089 integer (94 binary)

Root relaxation: objective 4.460600e+02, 640 iterations, 0.00 seconds (0.00 work units)

    Nodes 